In [17]:
#Importing modules
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers.experimental import preprocessing
import math
# tf.keras.mixed_precision.set_global_policy('mixed_float16')
# tf.keras.backend.floatx()
# import pillow

In [18]:
import tensorflow as tf

tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [19]:
TRAIN_DATA_DIR="D:\Codes\Image-Classification\cardataset/train"
VALIDATION_DATA_DIR="D:\Codes\Image-Classification\cardataset/val"
TRAIN_SAMPLES = 500
VALIDATION_SAMPLES = 500
NUM_CLASSES = 17
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

In [20]:
train_datagen = ImageDataGenerator(#preprocessing_function=preprocessing,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)
val_datagen = ImageDataGenerator()#preprocessing_function=preprocessing)

In [21]:
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR,
                                                    target_size=(IMG_WIDTH,
                                                                 IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

Found 22852 images belonging to 17 classes.
Found 5193 images belonging to 17 classes.


In [22]:
def model_maker():
    base_model = EfficientNetB0(include_top=False,
                           input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    # x=tf.cast(input,tf.float32)
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

In [23]:
model = model_maker()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['acc'])
model.fit(
    train_generator,
    steps_per_epoch=math.ceil(18),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE))

Epoch 1/10
18/18 [==============================] - 30s 2s/step - loss: 2.3856 - acc: 0.3030 - val_loss: 1.8667 - val_acc: 0.5547
Epoch 2/10
18/18 [==============================] - 27s 1s/step - loss: 2.0826 - acc: 0.3845 - val_loss: 1.9216 - val_acc: 0.4121
Epoch 3/10
18/18 [==============================] - 28s 2s/step - loss: 1.8272 - acc: 0.4323 - val_loss: 1.7211 - val_acc: 0.5215
Epoch 4/10
18/18 [==============================] - 27s 2s/step - loss: 1.9117 - acc: 0.4470 - val_loss: 1.5001 - val_acc: 0.5977
Epoch 5/10
18/18 [==============================] - 38s 2s/step - loss: 1.8492 - acc: 0.4227 - val_loss: 1.6345 - val_acc: 0.5742
Epoch 6/10
18/18 [==============================] - 38s 2s/step - loss: 1.6948 - acc: 0.4618 - val_loss: 1.5397 - val_acc: 0.5879
Epoch 7/10
18/18 [==============================] - 39s 2s/step - loss: 1.7222 - acc: 0.4809 - val_loss: 1.6577 - val_acc: 0.5645
Epoch 8/10
18/18 [==============================] - 37s 2s/step - loss: 1.7708 - acc: 0.46

In [24]:
class_names=list((validation_generator.class_indices).keys())
class_names

['Ambulance',
 'Barge',
 'Bicycle',
 'Boat',
 'Bus',
 'Car',
 'Cart',
 'Caterpillar',
 'Helicopter',
 'Limousine',
 'Motorcycle',
 'Segway',
 'Snowmobile',
 'Tank',
 'Taxi',
 'Truck',
 'Van']

In [25]:
model.save('model_EfficientNetB0')

INFO:tensorflow:Assets written to: model_MobileNet_preprocess\assets


In [26]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
model= tf.keras.models.load_model("D:\Codes\Image-Classification\model_EfficientNetB0")


In [33]:
img_path = "D:\Codes\Image-Classification/airbus-helicopters-4-gq-3jun15-pr_b.jpg" #bike
# img_path='D:\Codes\Image-Classification\elantra-1080p.jpg' #elantra
# img=image.load_img(img_path)
 
img = tf.io.read_file(img_path)
img = tf.image.decode_jpeg(img)
img = tf.image.resize(img, [224,224])
# img=preprocess_input(img)
img= tf.cast(tf.expand_dims(img, axis=0), tf.int16)
#  
 
# img = tf.image.decode_jpeg(img)
# img = tf.image.resize(img, [224,224])
# img= tf.expand_dims(img, axis=0)
# img=tf.cast(img, tf.int16)
# img=preprocess_input(img)
  

prediction=model.predict(img)



class_names[tf.argmax(prediction[0])]
(tf.reduce_max(prediction)*100).numpy()


23.155386

In [28]:
print(img
)

tf.Tensor(
[[[[104  88  56]
   [106  94  62]
   [110  99  58]
   ...
   [115 106  92]
   [111 104  85]
   [110 102  80]]

  [[115 100  64]
   [119 102  68]
   [132 112  75]
   ...
   [110 102  83]
   [112 106  82]
   [102  97  81]]

  [[125 111  69]
   [135 117  79]
   [140 120  85]
   ...
   [ 80  77  68]
   [ 83  81  70]
   [ 74  74  67]]

  ...

  [[ 40  49  21]
   [ 72  84  43]
   [ 33  40  24]
   ...
   [ 26  41  41]
   [ 21  36  32]
   [ 36  45  35]]

  [[ 59  71  34]
   [ 50  67  15]
   [ 37  45  28]
   ...
   [ 30  41  39]
   [ 45  54  43]
   [ 61  66  46]]

  [[ 68  81  43]
   [ 46  58  21]
   [ 37  47  20]
   ...
   [ 36  43  38]
   [ 60  70  49]
   [ 73  75  58]]]], shape=(1, 224, 224, 3), dtype=int16)


In [29]:
validation_generator.class_indices.keys()

dict_keys(['Ambulance', 'Barge', 'Bicycle', 'Boat', 'Bus', 'Car', 'Cart', 'Caterpillar', 'Helicopter', 'Limousine', 'Motorcycle', 'Segway', 'Snowmobile', 'Tank', 'Taxi', 'Truck', 'Van'])

In [30]:
from tensorflow.keras.preprocessing import image
def make_prediction(img):#,class_names):
    img=image.load_img(img, target_size=(224,224))

    img_array = image.img_to_array(img)
    expand_img_array=np.expand_dims(img_array, axis=0)
    # preprocessed_img=preprocess_input(expand_img_array

    prediction=model.predict(preprocessed_img)


    pred_prob=class_names[tf.argmax(prediction[0])]
    pred_class=(tf.reduce_max(prediction)*100).numpy()
    return pred_class, pred_prob

In [31]:
img = "D:\Codes\Image-Classification\elantra-1080p.jpg"

In [32]:
make_prediction(img)

NameError: name 'preprocessed_img' is not defined